## Setting Up Libraries and Packages

In [ ]:
!pip install web3
!pip install --force-reinstall jsonschema==3.2.0
exit()

In [ ]:
!pip install kora -q

In [ ]:
from kora import drive
drive.link_nbs()

In [ ]:
from flutils import *

In [ ]:
from SmartContractInteraction import *

## Making Clients (registering them to the smart contract)

In [ ]:
for i in range(NUM_ACCOUNTS):
  print('Creating Client', i+1)
  make_me_client(acc_add[i], acc_pk[i])

In [ ]:
data_request(acc_add[0], acc_pk[0])

AttributeDict({'transactionHash': HexBytes('0xa704312790a193634611ccde14cf23abb87c80ed29b06f3f391fdb6d1c1e7dfc'), 'transactionIndex': 0, 'blockHash': HexBytes('0x0f9749058754f6c49d844fb93c4b21634c4d400df95a3575936f65a037ab3a81'), 'blockNumber': 203, 'from': '0xe3058e84df37C4404bDBF2467Ed0b21bF82489Ed', 'to': '0x56A84Ba5A9BACa1f3aD79da38c0e2B4efF624273', 'gasUsed': 62107, 'cumulativeGasUsed': 62107, 'contractAddress': None, 'logs': [], 'status': 1, 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000')})


## Trainer Clients Posting data after training

## Aggregator Clients

In [ ]:
for i in range(len(account_addresses)-1, -1, -1):
  print('Client', i+1, 'aggregating data')
  model_aggregation(account_addresses[i], private_keys[i])

In [ ]:
import numpy as np
import random
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from tqdm import tqdm
import time

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers.legacy import SGD
from tensorflow.keras import backend as K

## Loading and Processing Dataset

In [ ]:
data_path = '/content/drive/MyDrive/Datasets/swarm_aligned'

In [ ]:
#apply our function
data_list, label_list = load(data_path)
labels = list(set(label_list.tolist())) #unique labels

#binarize the labels
#lb = LabelBinarizer()
#label_list = lb.fit_transform(label_list)
n_values = np.max(label_list) + 1
label_list = np.eye(n_values)[label_list]

In [ ]:
#split data into training and test set
X_train, X_test, y_train, y_test = train_test_split(data_list,
                                                    label_list,
                                                    test_size=0.2,
                                                    random_state=7)

## Creating Clients to Participate in FL

In [ ]:
#create clients
clients = create_clients(X_train, y_train, num_clients=100, initial='client')

In [ ]:
#process and batch the training data for each client
clients_batched = dict()
for (client_name, data) in clients.items():
    clients_batched[client_name] = batch_data(data)

#process and batch the test set
test_batched = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(len(y_test))

comms_round = 5 #number of global epochs


## Creating Optimizer (To Perform Gradient Descent)

In [ ]:
#create optimizer
lr = 0.01
loss='categorical_crossentropy'
metrics = ['accuracy']
optimizer = SGD(learning_rate=lr,
                decay=lr / comms_round,
                momentum=0.9
               )

#initialize global model
#print(data_list.shape,labels)
smlp_global = SimpleMLP()
global_model = smlp_global.build(data_list.shape[1],len(labels))

In [ ]:
print(global_model.get_weights())

In [ ]:
list(clients_batched['client_1'])[0][0].shape[0]

32

## Training and Averaging

In [ ]:
#commence global training loop
for comm_round in range(comms_round):

    # get the global model's weights - will serve as the initial weights for all local models
    global_weights = global_model.get_weights()

    #initial list to collect local model weights after scalling
    scaled_local_weight_list = list()

    #randomize client data - using keys
    client_names= list(clients_batched.keys())
    random.shuffle(client_names)

    #loop through each client and create new local model
    # i = 1
    for client in tqdm(client_names , desc = 'Progress Bar'):
        #time.sleep(0.5)
        smlp_local = SimpleMLP()
        local_model = smlp_local.build(data_list.shape[1],len(labels))
        local_model.compile(loss=loss,
                      optimizer=optimizer,
                      metrics=metrics)

        #print(local_model.summary())
        #print(clients_batched)
        #set local model weight to the weight of the global model
        local_model.set_weights(global_weights)

        #fit local model with client's data
        local_model.fit(clients_batched[client], epochs=1, verbose=0)

        #scale the model weights and add to list
        scaling_factor = weight_scalling_factor(clients_batched, client)
        scaled_weights = scale_model_weights(local_model.get_weights(), scaling_factor)
        scaled_local_weight_list.append(scaled_weights)
        # print('Client',i, 'is posting data')
        # i += 1
        # data_post(acc_add[i], acc_pk[i], scaled_weights)

        #clear session to free memory after each communication round
        K.clear_session()

    #update global model
    average_weights = sum_scaled_weights(scaled_local_weight_list)
    global_model.set_weights(average_weights)

    #test global model and print out metrics after each communications round
    for(X_test, Y_test) in test_batched:
        global_acc, global_loss = test_model(X_test, Y_test, global_model, comm_round)


Progress Bar: 100%|██████████| 100/100 [01:15<00:00,  1.33it/s]


151/151 [==============================] - 1s 7ms/step
comm_round: 0 | global_acc: 68.901% | global_loss: 0.6840079426765442


Progress Bar: 100%|██████████| 100/100 [01:07<00:00,  1.49it/s]


151/151 [==============================] - 1s 7ms/step
comm_round: 1 | global_acc: 68.901% | global_loss: 0.6785796284675598


Progress Bar:  20%|██        | 20/100 [00:13<00:50,  1.59it/s]

In [ ]:
# NTS: TRY TO CHANGE THE BATCH SIZE TO 3200 I.E. 32*100 FOR 100 CLIENTS.
SGD_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(len(y_train)).batch(320)
smlp_SGD = SimpleMLP()
SGD_model = smlp_SGD.build(data_list.shape[1], len(labels))

SGD_model.compile(loss=loss,
              optimizer=optimizer,
              metrics=metrics)

In [ ]:
# fit the SGD training data to model
_ = SGD_model.fit(SGD_dataset, epochs=100, verbose=0)

In [ ]:
#test the SGD global model and print out metrics
for(X_test, Y_test) in test_batched:
        SGD_acc, SGD_loss = test_model(X_test, Y_test, SGD_model, 1)

76/76 [==============================] - 1s 7ms/step
comm_round: 1 | global_acc: 69.734% | global_loss: 0.6598128080368042
